In [1]:
# установка абсолютного пути к проекту
# не нужно в случае `pip install`
%run -i tools/setup_env.py

## Примеры использования `torchcnnbuilder.preprocess`

Этот подмодуль содержит функции предобработки тензоров. Представлены функции для разделения многомерных тензоров на пространственно-временные ряды с частями X и Y, поскольку основная функциональность разработана для задачи прогнозирования N-мерных временных рядов

### Подмодуль `torchcnnbuilder.preprocess.time_series`

Сначала создадим синтетические данные. Скрипт генерации находится в `../tools/generating_time_series.py`. Данные состоят из 210 numpy матриц 100x100, которые формируют 2-мерный временной ряд - движение квадрата по кругу. Для визуальной демонстрации пространственно-временного ряда ниже приложена покадровая анимация:
<img src="../tools/media/time_series_animation.gif" alt="animation" style="width:40%; display: block; margin-left: auto; margin-right: auto;">

In [1]:
%%capture
from examples.tools.generating_time_series import synthetic_time_series

# первый объект - это класс анимации всего временного ряда
_, data = synthetic_time_series()

In [2]:
print(f'Длина набора данных: {len(data)}, Форма одной матрицы: {data[0].shape}')

Длина набора данных: 210, Форма одной матрицы: (100, 100)


#### Функция `single_output_tensor`

In [3]:
from torchcnnbuilder.preprocess import single_output_tensor

Параметры:

- **data**: N-мерные массивы, списки, numpy массивы, тензоры и т.д.
- **forecast_len**: длина прогноза для каждого y-train будущего тензора (цель)
- **additional_x**: дополнительные x-train данные. По умолчанию: None
- **additional_is_array**: если дополнительный x-train является массивом данных x_i, как другие временные ряды. По умолчанию: False
- **additional_x_stack**: если True, объединяет каждый additional_x_i с x-train. По умолчанию: True
- **threshold**: порог бинаризации для каждого y-тензора. По умолчанию: False
- **x_binarize**: бинаризация с порогом для каждого x-тензора. По умолчанию: False

Возвращает:
TensorDataset из X-train и y-train

Эта функция предобрабатывает n-мерный временной ряд в тензор только с частями X и Y и возвращает `TensorDataset`. Допустим, мы хотим предсказать следующие 30 состояний на основе остальных данных, тогда `forecast_len=30`. Функция может работать со всеми данными типа последовательности или массива, но библиотека не использует никаких зависимостей, кроме `torch`, поэтому в случае `numpy.array` вы можете получить подобное предупреждение, поскольку внутри функции происходит преобразование в тензор. Визуализировать результат функции можно следующим образом:

<img src="../tools/media/single_output_tensor.png" alt="single tensor" style="width:70%; display: block; margin-left: auto; margin-right: auto;">

In [4]:
dataset = single_output_tensor(data=data, 
                               forecast_len=30)

C:\Users\Julia\Documents\NSS_lab\TorchCNNBuilder\torchcnnbuilder\preprocess\_dynamic_window.py:47: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:277.)
  tensors = torch.Tensor(data)


In [5]:
# проверка размеров данных
for batch in dataset:
    print(f'Форма X: {batch[0].shape}\nФорма Y: {batch[1].shape}')

Форма X: torch.Size([180, 100, 100])
Форма Y: torch.Size([30, 100, 100])


Если вы хотите делать прогноз с учетом дополнительных данных, то вы можете добавить дополнительный X с помощью `additional_x`. В этом случае два X-массива объединяются в один тензор: появится дополнительное измерение *(в нашем случае канал)* после значения `X.shape[0]`

In [6]:
dataset = single_output_tensor(data=data,
                               additional_x=data.copy(),
                               forecast_len=30)

In [8]:
for batch in dataset:
    print(f'новая объединённая размерность X: {batch[0].shape}\nРазмерность Y: {batch[1].shape}') 

новая объединённая размерность X: torch.Size([180, 2, 100, 100])
Размерность Y: torch.Size([30, 100, 100])


Но вы можете избежать объединения двух разных X, изменив параметр `additional_x_stack` на `False` *(по умолчанию `True`)* и получить два отдельных X

In [10]:
dataset = single_output_tensor(data=data,
                               additional_x=data.copy(),
                               additional_x_stack=False,
                               forecast_len=30)

In [11]:
for batch in dataset:
    print(f'Размерность X1: {batch[0].shape}\nРазмерность X2: {batch[1].shape}\nРазмерность Y: {batch[2].shape}') 

Размерность X1: torch.Size([180, 100, 100])
Размерность X2: torch.Size([180, 100, 100])
Размерность Y: torch.Size([30, 100, 100])


Если вы хотите создать набор данных из нескольких X *(более 2)*, то используйте следующий шаблон с параметром `additional_is_array=True`, в этом случае все X будут объединены в новом измерении *(только такое поведение поддерживается при использовании множественных X)*

In [12]:
dataset = single_output_tensor(data=data,
                               additional_x=[data.copy(), data.copy(), data.copy()],
                               additional_is_array=True,
                               forecast_len=30)

In [13]:
for batch in dataset:
    print(f'новая объединённая Размерность X: {batch[0].shape}\nРазмерность Y: {batch[1].shape}')  

новая объединённая Размерность X: torch.Size([180, 4, 100, 100])
Размерность Y: torch.Size([30, 100, 100])


Вы также можете использовать `threshold` для бинаризации ваших данных. По умолчанию бинаризация происходит только для Y, но это также можно сделать для X, используя параметр `x_binarize=True` *(все X или новый объединённый X будут бинаризованы)*

In [14]:
import numpy as np

# матрица гауссовского шума
gaussian_noise_matrix = np.random.normal(loc=0, scale=1, size=(100, 100))
noise_data = data - gaussian_noise_matrix

print(f'максимум данных: {noise_data.max()} | минимум: {noise_data.min()}')

максимум данных: 4.786741031089559 | минимум: -4.2005561798251305


In [15]:
dataset = single_output_tensor(data=noise_data,
                               additional_x=[noise_data.copy(), noise_data.copy(), noise_data.copy()],
                               additional_is_array=True,
                               forecast_len=30,
                               threshold=0.5,
                               x_binarize=True)

In [16]:
for batch in dataset:
    print(f'новая объединённая Размерность X: {batch[0].shape}\nРазмерность Y: {batch[1].shape}',
          f'новый объединённый X макс: {batch[0].max()} | мин: {batch[0].min()}\nY макс: {batch[1].max()} | мин: {batch[1].min()}',
          sep='\n\n') 

новая объединённая Размерность X: torch.Size([180, 4, 100, 100])
Размерность Y: torch.Size([30, 100, 100])

новый объединённый X макс: 1.0 | мин: 0.0
Y макс: 1.0 | мин: 0.0


#### Функция `multi_output_tensor`

In [17]:
from torchcnnbuilder.preprocess import multi_output_tensor

Параметры:

- **data**: N-мерные массивы, списки, numpy массивы, тензоры и т.д.
- **forecast_len**: длина прогноза для каждого y-train будущего тензора (цель)
- **pre_history_len**: длина предыстории для каждого x-train будущего тензора
- **additional_x**: дополнительные x-train данные. По умолчанию: None
- **additional_is_array**: если дополнительный x-train является массивом данных x_i, как другие временные ряды. По умолчанию: False
- **additional_x_stack**: если True, объединяет каждый additional_x_i с x-train. По умолчанию: True
- **threshold**: порог бинаризации для каждого y-тензора. По умолчанию: False
- **x_binarize**: бинаризация с порогом для каждого x-тензора. По умолчанию: False

Возвращает: TensorDataset из X-train и y-train

Эта функция предобрабатывает n-мерный временной ряд в тензор  **скользящим окном** для частей X и Y и возвращает `TensorDataset`. В дополнение к диапазону прогнозирования `forecast_len`, теперь вам нужно указать период, на основе которого мы прогнозируем данные, т.е. выбрать параметр `pre_history_len` *(что означает, что можно бинаризовать и объединять различные X-окна)*. В остальном вся функциональность остается такой же, как у функции `single_output_tensor`. Такое поведение функции позволяет обучать и прогнозировать за заданный период времени, а не на всем наборе данных сразу

Предположим, что мы будем прогнозировать 30 кадров нашего 2-мерного временного ряда на основе предыдущих 60. Визуализировать результат функции можно следующим образом:
<img src="../tools/media/multi_output_tensor.png" alt="single tensor" style="width:70%; display: block; margin-left: auto; margin-right: auto;">

In [18]:
dataset = multi_output_tensor(data=data, 
                              forecast_len=30,
                              pre_history_len=60)

In [19]:
for i, batch in enumerate(dataset):
    print(f'номер батча: {i}',
          f'Размерность X: {batch[0].shape}\nРазмерность Y: {batch[1].shape}',
          sep='\n',
          end='\n\n')
    if i == 1:
        break
print(f'Длина набора данных (количество батчей/X-окон): {len(dataset)}')

номер батча: 0
Размерность X: torch.Size([60, 100, 100])
Размерность Y: torch.Size([30, 100, 100])

номер батча: 1
Размерность X: torch.Size([60, 100, 100])
Размерность Y: torch.Size([30, 100, 100])

Длина набора данных (количество батчей/X-окон): 120


Использование `threshold` и нескольких X, как в последнем примере предыдущей функции. Теперь каждый X должен иметь 4 канала, потому что мы используем всего 4 различных 2-мерных временных ряда

In [20]:
dataset = multi_output_tensor(data=noise_data,
                              additional_x=[noise_data.copy(), noise_data.copy(), noise_data.copy()],
                              additional_is_array=True,
                              forecast_len=30,
                              pre_history_len=60,
                              threshold=0.5,
                              x_binarize=True)

In [21]:
for i, batch in enumerate(dataset):
    print(f'номер батча: {i}',
          f'новая объединённая Размерность X: {batch[0].shape}\nРазмерность Y: {batch[1].shape}',
          f'новый объединённый X макс: {batch[0].max()} | мин: {batch[0].min()}\nY макс: {batch[1].max()} | мин: {batch[1].min()}',
          sep='\n',
          end='\n\n')
    if i == 1:
        break
print(f'Длина набора данных (количество батчей/X-окон): {len(dataset)}')

номер батча: 0
новая объединённая Размерность X: torch.Size([60, 4, 100, 100])
Размерность Y: torch.Size([30, 100, 100])
новый объединённый X макс: 1.0 | мин: 0.0
Y макс: 1.0 | мин: 0.0

номер батча: 1
новая объединённая Размерность X: torch.Size([60, 4, 100, 100])
Размерность Y: torch.Size([30, 100, 100])
новый объединённый X макс: 1.0 | мин: 0.0
Y макс: 1.0 | мин: 0.0

Длина набора данных (количество батчей/X-окон): 120
